In [ ]:
library(hash)
library(xts)
library(lubridate)
library(forecast)
library(fpp)

In [ ]:
DATA_FOLDER <- "/home/samarth/workspaces/datakind-workspace/analytics/time-series/data/topNComplaints"

In [ ]:
loadData <- function(dataFolder) {
    files <- list.files(dataFolder)
    data <- list()
    for(file in files) {    
        df <- read.csv(paste0(dataFolder, "/", file), stringsAsFactors=F)    
        minYear <- min(df$Year)
        complaintType <- substr(file,1,(nchar(file))-4)    
        tsObject <- ts(df$Complaints, start=c(minYear, 1), frequency = 12)
        data[[complaintType]] <- tsObject
    }
    data
}
data <- loadData(DATA_FOLDER)
complaintTypes <- names(data)
data[[complaintTypes[1]]]

In [ ]:
buildArimaModel <- function(data, complaintType, order=c(12, 1, 1)) {
    print(paste0("Building ARIMA model for ", complaintType))
    raw <- data[[complaintType]]        
    # since pre-2012, there were very few data points, subset it. also, we only consider 2015 data    
    monthly <- window(raw, start=c(2012,1), end=c(2015, 12))    
    trainStart <- c(2012, 1)
    trainEnd <- c(2015, 1)
    testStart <- trainEnd
    testEnd <- c(2015, 12)    
    # split into 'test' and 'train' set
    trainData <- window(monthly, start=trainStart, end=trainEnd)
    testData <- window(monthly, start=testStart, end=testEnd)
    # plot acf and pacf    
    tsdisplay(trainData, main="Plot of data")            
    fit <- Arima(trainData, order=order, method="ML")    
    plot(forecast(fit, h=12), main="Forecasts")
    lines(testData, lty=2)
}
buildArimaModel(data, complaintTypes[10], order=c(12, 2, 1))

In [ ]:
expand.grid(p=seq(1,12), q=seq(1, 12), d=seq(1, 12))